In [1]:
%%capture
!pip install import_ipynb --no-cache
import import_ipynb
m = __import__("Methods")

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
from sklearn.preprocessing import StandardScaler
import numpy as np
import ast

In [3]:
results_df = pd.read_csv("data/results/all_model_results_n1500.csv")

results_df

,Model,Target Variable,MSE,RMSE,MAE,R2,R,Selected Features,Top 20 SHAP Values
0,ANN_Final_Test,th_positive_cells,2360.669445,48.586721,26.887698,-0.610824,NaN,"FBgn0015774,FBgn0015776,FBgn0015777,FBgn001577...","{'FBgn0023496': 0.04475578278790186, 'FBti0063..."
1,Linear_Regression_Final_Test,th_positive_cells,1977.321753,44.467086,24.632739,-0.349243,NaN,"FBgn0000455,FBgn0000459,FBgn0000463,FBgn000055...","{'FBgn0036833': 0.19138259979406097, 'FBgn0264..."
2,Ridge_Final_Test,th_positive_cells,1992.185801,44.633909,24.704676,-0.359386,NaN,"FBgn0000032,FBgn0000061,FBgn0000075,FBgn000011...","{'FBgn0036833': 0.058198007816529104, 'FBgn026..."
3,Lasso_Final_Test,th_positive_cells,2025.620097,45.006889,25.037738,-0.382200,NaN,"FBgn0000003,FBgn0000008,FBgn0000014,FBgn000001...","{'FBgn0267766': 0.5214049144977432, 'FBgn02679..."
4,SVM_Final_Test,th_positive_cells,2002.409180,44.748287,24.775086,-0.366362,NaN,"FBgn0000032,FBgn0000075,FBgn0000116,FBgn000012...","{'FBgn0036833': 0.05154506209405348, 'FBgn0264..."
5,ANN_Final_Test,repo_glial_cells,50968.477258,225.761993,104.357988,-0.280938,NaN,"FBgn0015737,FBgn0015754,FBgn0015756,FBgn001576...","{'FBgn0022700': 0.07558726538854034, 'FBti0062..."
6,ANN_Final_Test,repo_glial_cells,50564.206508,224.864863,103.830265,-0.270778,NaN,"FBgn0015773,FBgn0015774,FBgn0015776,FBgn001577...","{'FBgn0025616': 0.05832820848762684, 'FBgn0023..."
7,Linear_Regression_Final_Test,repo_glial_cells,50474.033893,224.664269,103.663537,-0.268512,NaN,"FBgn0000404,FBgn0000463,FBgn0000535,FBgn000056...","{'FBgn0054002': 0.044811555513037386, 'FBgn003..."
8,Ridge_Final_Test,repo_glial_cells,50542.002066,224.815484,103.737821,-0.270220,NaN,"FBgn0000003,FBgn0000008,FBgn0000032,FBgn000004...","{'FBgn0032658': 0.03919693624926304, 'FBgn0015..."
9,Lasso_Final_Test,repo_glial_cells,50258.259705,224.183540,103.226096,-0.263089,NaN,"FBgn0000003,FBgn0000008,FBgn0000017,FBgn000002...","{'FBgn0000003': 0.0, 'FBgn0000008': 0.0, 'FBgn..."


In [12]:
ANN_features = results_df.iloc[0]["Top 20 SHAP Values"]
ANN_features = ast.literal_eval(ANN_features)
df = pd.DataFrame(list(ANN_features.items()), columns=['Gene', 'SHAP Value'])

In [14]:
# Import the MyGeneInfo module
mg = m.mygene.MyGeneInfo()

# Get the list of FlyBase gene IDs from your data
flybase_ids = list(df["Gene"])

# Query MyGeneInfo for FlyBase IDs
ginfo = mg.querymany(flybase_ids, scopes='flybase')

# Initialize lists to keep track of genes to drop and those with symbols
drop_list = []
hit_list = []

# Iterate through the query results
for g in ginfo:
    # Check if the gene has a 'symbol'; if not, add to the drop list
    if 'symbol' not in g:
        drop_list.append(g['query'])
    else:
        # Check if the gene symbol is already in the hit list; if not, add it
        if g['symbol'] not in hit_list:
            # Replace the FlyBase ID with its corresponding gene symbol
            df.loc[df['Gene'] == g['query'], 'Gene'] = g['symbol']
            hit_list.append(g['symbol'])

# Print the shape of the RNA-seq data before filtering
print('RNA-seq data shape before: ', df.shape)

# Remove rows where the gene ID did not have a corresponding symbol
df = df[~df['Gene'].isin(drop_list)]

# Print the shape of the RNA-seq data after filtering
print('RNA-seq data shape after: ', df.shape)

13 input query terms found no hit:	['FBti0062398', 'FBti0062381', 'FBti0062695', 'FBti0063086', 'FBti0063079', 'FBti0064274', 'FBti0062


RNA-seq data shape before:  (20, 2)
RNA-seq data shape after:  (7, 2)


In [23]:
print(df["Gene"].to_list())

['Lip1', 'Ugt37B1', 'Pk34A', 'CG13359', 'Lectin-galC1', 'Or33b', 'ind']


In [16]:
# look at the names of the libraries that are available to use for PEA and GSEA
gene_sets = m.gp.get_library_name(organism="fly")

In [18]:
# Perform enrichment analysis with FlyBase genes using Enrichr
enr = m.gp.enrichr(
    gene_list=df["Gene"].to_list(),  # List of FlyBase gene symbols
    gene_sets=gene_sets,
    organism='fly',  # Set to 'fly' for Drosophila (FlyBase genes)
    outdir=None
)

,Term,Genes,Adjusted P-value
0,apterous-expressing neuron of the lateral clus...,ind,0.007866
1,apterous-expressing neuron of the lateral clus...,ind,0.007866
24,ventral_nerve_cord_primordium,ind,0.011188
284,holo TFIIH complex (GO:0005675),Pk34A,0.025158
293,galactose binding (GO:0005534),Lectin-galC1,0.018852
294,cyclin-dependent protein kinase activity (GO:0...,Pk34A,0.018852
295,cyclin-dependent protein serine/threonine kina...,Pk34A,0.018852
296,lipase activity (GO:0016298),Lip1,0.042963
297,olfactory receptor activity (GO:0004984),Or33b,0.044263
310,protein_tyrosine_kinase_binding_(GO:1990782),ind,0.028695


In [21]:
pd.set_option('display.max_colwidth', None)
enr.results[enr.results['Adjusted P-value'] < 0.05][["Genes", 'Term']]

,Genes,Term
0,ind,apterous-expressing neuron of the lateral cluster of the thorax Ap4
1,ind,apterous-expressing neuron of the lateral cluster of the thorax
24,ind,ventral_nerve_cord_primordium
284,Pk34A,holo TFIIH complex (GO:0005675)
293,Lectin-galC1,galactose binding (GO:0005534)
294,Pk34A,cyclin-dependent protein kinase activity (GO:0097472)
295,Pk34A,cyclin-dependent protein serine/threonine kinase activity (GO:0004693)
296,Lip1,lipase activity (GO:0016298)
297,Or33b,olfactory receptor activity (GO:0004984)
310,ind,protein_tyrosine_kinase_binding_(GO:1990782)
